In [1]:
from selenium.webdriver.common.by import By
import selenium as sel
import pandas as pd
import time
import datetime

import os as os
import csv

In [39]:
empresa = input("Digite la empresa a extraer: ")

In [40]:
def verif(u):
    with open('../data/raw/articulos.csv', encoding="utf8") as f:
        csvreader = csv.reader(f, delimiter=",")
        for row in csvreader:
            if u in row[5]:
                 return False
        return True


In [41]:
#cerar driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = sel.webdriver.Edge()
driver.get(f'https://www.larepublica.co/{empresa}')
time.sleep(2)

In [42]:
articulos = driver.find_elements(By.XPATH,'.//div[contains(@class,"row news")]')

In [43]:
titulares = []
for art in articulos:
    url = art.find_elements(By.XPATH,'.//a')[1].get_attribute('href')
    if verif(url):
        fechaP = art.find_element(By.XPATH,'.//span[@class = "date-news"]').text
        tema = art.find_elements(By.XPATH,'.//a')[1].text
        resumen = art.find_element(By.XPATH,'.//p').text
        titulo = art.find_element(By.XPATH,'.//h2').text
        imagen = art.find_elements(By.XPATH,'.//img')[0].get_attribute('src')
        titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        'Titulo':titulo,
                        'Fecha Publicacion':fechaP,
                        'Tema':tema,
                        'Resumen':resumen,
                        'URL':url,
                        'Imagen':imagen})
    

In [44]:
len(titulares)

22

In [45]:
for tit in titulares:
    driver.get(tit['URL'])
    try :
        secAutor = driver.find_element(By.XPATH,'.//div[contains(@class,"author-article")]')
    except:
        autor = 'SIN AUTOR'
    else:
        try:
            autor = secAutor.find_element(By.XPATH,'.//button').text
        except:
            autor = secAutor.find_element(By.XPATH,'.//span').text

    ## agregar autor
    tit['Autor'] = autor
    try :
        html = driver.find_element(By.XPATH,'.//div[contains(@class,"html-content")]')
        parrafos = html.find_elements(By.XPATH,'.//p')
    except:
        parrafos = 'SIN PARRAFOS'
    else:
        contenido = ''
        for i in parrafos:
            contenido += i.text
        ## Agregar contenido
        tit['Contenido'] = contenido
    try :
        related = driver.find_elements(By.XPATH,'.//div[contains(@class,"relatedNews")]')
        relNewsUrls = []
        for i in related:
            relNewsUrls.append(i.find_element(By.XPATH,'.//a').get_attribute('href'))
    except:
        relNewsUrls = []

    ##agregar lista de URLs de noticias relacionadas
    tit['RelNewsUrls'] = relNewsUrls     

In [46]:
print(titulares[-1])

{'Fecha Extraccion': '2022-08-31 15:15:42', 'Titulo': 'Ecopetrol anunció el descubrimiento de extensión de la gasífera Gorgon-2', 'Fecha Publicacion': '10/08/2022', 'Tema': 'ENERGÍA', 'Resumen': 'El pozo se encuentra ubicado aproximadamente a 70 kilómetros de la costa caribe y fue perforado con un buque de última tecnología para aguas profundas', 'URL': 'https://www.larepublica.co/economia/ecopetrol-anuncio-el-descubrimiento-de-extension-de-la-provincia-gasifera-gorgon-2-3422266', 'Imagen': "data:image/svg+xml,%3Csvg xmlns='http://www.w3.org/2000/svg' viewBox='0 0 1080 900'%3E%3C/svg%3E", 'Autor': 'JUAN PABLO VARGAS CUELLAR', 'Contenido': 'Ecopetrol anunció que con la perforación del pozo Gorgon-2, en aguas ultra profundas en el sur del Caribe colombiano, se comprobó la presencia de gas. Se trata de la perforación de mayor profundidad, la cual supera los 4.000 metros aproximadamente.El pozo se encuentra ubicado a unos 70 kilómetros de la costa caribe y fue perforado con un buque de últ

In [47]:
columnas = ['Fecha Extraccion','Titulo', 'Fecha Publicacion','Tema','Resumen','URL','Imagen','Autor','Contenido','RelNewsUrls']

In [49]:
with open('../data/raw/articulos.csv', 'a', newline='') as csv_file:
    dict_object = csv.DictWriter(csv_file, fieldnames=columnas) 
  
    dict_object.writerows(titulares)

In [38]:
pd.json_normalize(titulares).to_csv(f'../data/raw/articulos.csv',index=False)